# 🛡️ AER - Smart Access Review Generator (v6.5 Auto-Resolve)

**New Features in v6.5:**
1.  **Missing Email Handler**: Automatically detects rows with missing emails.
2.  **Global Directory Sync**: Pre-fetches all active users (~1600) for instant name-to-email lookup.
3.  **Resolver UI**: Interactive widget to fix missing emails before processing.

**Standard Features (v6.4):**
- Enhanced Reviewer Mapping & Smart Column Detection
- Branch Support & Visual Enhancements
- Excel Validation & Status Logic

In [ ]:
# === Cell 1: Setup & Engine (v6.5 Directory Sync) ===
import os, sys, logging, glob, io, re, requests, json
import pandas as pd
import ipywidgets as widgets
from datetime import datetime
from dotenv import load_dotenv
from msal import PublicClientApplication
from concurrent.futures import ThreadPoolExecutor
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import get_column_letter
from IPython.display import display, HTML, clear_output

# 1. Paths & Logging
today_str = datetime.now().strftime('%Y-%m-%d')
BASE_DIR = os.path.join("output", today_str, "create")
LOG_DIR = os.path.join(BASE_DIR, "logs")
MAPPING_DIR = os.path.join("input", "mapping")
os.makedirs(LOG_DIR, exist_ok=True)

log_file = os.path.join(LOG_DIR, f"aer_create_{today_str}_{datetime.now().strftime('%H%M')}.log")
logger = logging.getLogger("aer")
logger.handlers.clear()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
fh = logging.FileHandler(log_file, encoding="utf-8")
fh.setFormatter(formatter)
logger.addHandler(fh)
logger.addHandler(logging.StreamHandler(sys.stdout))

# 2. Auth & Cache
load_dotenv()
headers = {}
try:
    tid, cid = os.getenv("AZURE_TENANT_ID"), os.getenv("AZURE_CLIENT_ID")
    if tid and cid:
        app = PublicClientApplication(cid, authority=f"https://login.microsoftonline.com/{tid}")
        res = app.acquire_token_interactive(scopes=["User.Read.All"], prompt="select_account")
        if "access_token" in res:
            headers = {"Authorization": f"Bearer {res['access_token']}"}
            logger.info("Authentication Successful")
except Exception as e:
    logger.error(f"Auth Critical Error: {str(e)}")

ad_cache = {}
ad_directory_cache = {} # v6.5: Stores {name_lower: email} for all active users
session = requests.Session()
adapter = requests.adapters.HTTPAdapter(pool_connections=50, pool_maxsize=50)
session.mount('https://', adapter)

def fetch_full_directory():
    """ 
    v6.5 New Feature: 
    Downloads all active users (approx 1600) to build a local Name->Email map.
    This is used to auto-resolve missing emails in the input file.
    """
    global ad_directory_cache
    if ad_directory_cache: return # Already loaded
    
    if not headers:
        # Mock data for testing
        ad_directory_cache = {"mock user": "mock.user@example.com"}
        return

    logger.info("\ud83d\udce5 Pre-fetching Global Directory (Active Users)...")
    users_map = {}
    # Fetch only active users, get name and mail
    url = "https://graph.microsoft.com/v1.0/users?$filter=accountEnabled eq true&$select=displayName,mail,userPrincipalName&$top=999"
    
    try:
        while url:
            r = session.get(url, headers=headers, timeout=10)
            if r.status_code == 200:
                data = r.json()
                for u in data.get('value', []):
                    name = u.get('displayName')
                    # Prefer mail, fallback to UPN
                    email = u.get('mail') or u.get('userPrincipalName')
                    if name and email:
                        # Normalize name for better matching (lowercase, strip)
                        users_map[str(name).strip().lower()] = str(email).strip().lower()
                
                url = data.get('@odata.nextLink') # Pagination
            else:
                logger.error(f"Directory sync failed: {r.status_code}")
                break
        
        ad_directory_cache = users_map
        logger.info(f"\u2705 Global Directory Loaded: {len(ad_directory_cache)} users")
    except Exception as e:
        logger.error(f"Directory sync error: {str(e)}")

def fetch_ad(email):
    """ v5.7 Logic: Single user lookup """
    email = str(email).strip().lower()
    if not re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', email):
        return {"email": email, "status": "Invalid", "name": "N/A", "dept": "N/A", "active": "N/A"}
    
    if email in ad_cache: return ad_cache[email]
    
    if not headers: 
        return {"email": email, "status": "Mock", "name": "User", "dept": "Mock Dept", "active": True}
    
    try:
        url = f"https://graph.microsoft.com/v1.0/users/{email}?$select=displayName,department,accountEnabled"
        r = session.get(url, headers=headers, timeout=5)
        if r.status_code == 200:
            d = r.json()
            res = {
                "email": email, 
                "status": "Found", 
                "name": d.get("displayName", "N/A"), 
                "dept": d.get("department") or "N/A", 
                "active": d.get("accountEnabled", "N/A")
            }
        else:
            res = {"email": email, "status": "Not Found", "name": "N/A", "dept": "N/A", "active": "N/A"}
    except:
        res = {"email": email, "status": "Error", "name": "N/A", "dept": "N/A", "active": "N/A"}

    ad_cache[email] = res
    return res

def is_valid_reviewer_name(reviewer):
    if not reviewer or pd.isna(reviewer): return False
    reviewer_str = str(reviewer).strip().lower()
    invalid = ["nan", "none", "", "null", "()", "n/a", "na", "tbd", "pending"]
    if reviewer_str in invalid: return False
    if len(reviewer_str) < 2 or not any(c.isalpha() for c in reviewer_str): return False
    return True

In [ ]:
# === Cell 2: Enhanced Hybrid Logic (v6.5 with Auto-Resolve) ===

# --- 1. UI Styling ---
style_html = """
<style>
    .aer-row { border-bottom: 1px solid #e0e0e0; padding: 5px 0; align-items: center; }
    .aer-header { font-weight: bold; background-color: #f0f0f0; padding: 8px 0; border-bottom: 2px solid #ccc; }
    .aer-cell { overflow: hidden; white-space: nowrap; text-overflow: ellipsis; }
    .widget-label { font-size: 11px; font-weight: 600; }
    .widget-dropdown select, .widget-text input { font-size: 11px; }
</style>
"""
display(HTML(style_html))

# --- 2. Widgets ---
up_list = widgets.FileUpload(accept='.xlsx, .csv', description="1. User List", button_style='info')
txt_list = widgets.HTML(value="<i>No file selected</i>", layout=widgets.Layout(margin='0 10px'))
up_map = widgets.FileUpload(accept='.csv', description="2. Reviewer Map", button_style='info')

btn_process = widgets.Button(description="\ud83d\ude80 Step 1: Analyze", button_style='warning', layout=widgets.Layout(width='180px'))
btn_save = widgets.Button(description="\ud83d\udcbe Step 2: Save", button_style='success', layout=widgets.Layout(width='180px'), disabled=True)

# v6.5 New Widgets for Resolver
resolver_area = widgets.Output()
out_area = widgets.Output()

# Global State
review_registry = []
processed_df = None
current_fname = ""
mapping_data = {"emails": {}, "depts": {}, "all_depts": [], "all_branches": []}
match_stats = {"email_match": 0, "dept_match": 0, "no_match": 0, "email_invalid": 0}
pending_df = None # Holds the dataframe while waiting for user to fix emails
pending_col_map = {} # Holds column names

# --- 3. Helper Functions ---
def get_latest_map():
    try:
        files = glob.glob(os.path.join(MAPPING_DIR, "*.csv"))
        return max(files, key=os.path.getmtime) if files else None
    except:
        return None

def on_list_file_change(c):
    try:
        if up_list.value and len(up_list.value) > 0:
            fname = up_list.value[0]['name']
            txt_list.value = f"<b style='color:green;'>\u2705 Selected: {fname}</b>"
            resolver_area.clear_output() # Clear previous resolver if new file
            out_area.clear_output()
    except:
        pass

up_list.observe(on_list_file_change, 'value')

map_path = get_latest_map()
txt_map = widgets.HTML(value=f"<b style='color:green;'>\u2705 Default: {os.path.basename(map_path)}</b>" if map_path else "<i>No map found</i>", layout=widgets.Layout(margin='0 10px'))

def identify_columns_smart(df):
    if len(df.columns) < 2: return df.columns[0], df.columns[0]
    c0, c1 = df.columns[0], df.columns[1]
    sample = df.head(20).fillna('').astype(str)
    score_0 = sum(1 for x in sample[c0] if '@' in x and '.' in x)
    score_1 = sum(1 for x in sample[c1] if '@' in x and '.' in x)
    return (c0, c1) if score_0 >= score_1 else (c1, c0)

def detect_map_column(df, candidates):
    cols = [str(c).lower().strip() for c in df.columns]
    for cand in candidates:
        for i, c in enumerate(cols):
            if cand in c: return df.columns[i]
    return None

# --- 4. Logic: Analysis with Enhanced Features ---
def do_process(b):
    """ Entry point: Checks for missing emails first """
    global pending_df, pending_col_map, current_fname
    out_area.clear_output()
    resolver_area.clear_output()
    
    if not up_list.value or len(up_list.value) == 0:
        with out_area: print("\u274c Error: Please upload a user list.")
        return

    b.disabled = True
    try:
        # Load User List
        f_item = up_list.value[0]
        current_fname = f_item['name']
        df_u = pd.read_csv(io.BytesIO(f_item['content'])) if current_fname.endswith('.csv') else pd.read_excel(io.BytesIO(f_item['content']))
        col_email, col_name = identify_columns_smart(df_u)
        
        # v6.5: Check for missing emails
        missing_mask = df_u[col_email].isna() | (df_u[col_email].astype(str).str.strip() == '')
        missing_count = missing_mask.sum()
        
        if missing_count > 0:
            # Trigger Resolver Flow
            pending_df = df_u
            pending_col_map = {'email': col_email, 'name': col_name}
            show_resolver_ui(df_u[missing_mask], col_name, col_email)
            b.disabled = False
            return
        
        # If clean, proceed to main logic
        execute_main_logic(df_u, col_email, col_name)
        
    except Exception as e:
        with out_area: print(f"\u274c Error: {str(e)}")
        logger.error(f"Process error: {str(e)}", exc_info=True)
        b.disabled = False

def show_resolver_ui(missing_df, col_name, col_email):
    """ v6.5: Displays the widget to fix missing emails """
    with resolver_area:
        print(f"\u26a0\ufe0f Found {len(missing_df)} rows with missing emails. Syncing Directory...")
        fetch_full_directory() # Ensure we have the map
        
        print("\ud83d\udd27 Please verify/enter emails below:")
        
        resolver_widgets = []
        
        # Header
        header = widgets.HBox([
            widgets.Label("Name (from File)", layout=widgets.Layout(flex='1', font_weight='bold')),
            widgets.Label("Email (Auto-Suggested)", layout=widgets.Layout(flex='1', font_weight='bold')),
            widgets.Label("Status", layout=widgets.Layout(width='100px', font_weight='bold'))
        ], layout=widgets.Layout(border_bottom='2px solid #ccc', padding='5px'))
        display(header)
        
        for idx, row in missing_df.iterrows():
            raw_name = str(row[col_name]).strip()
            clean_name = raw_name.lower()
            
            # Try to find in directory
            suggested_email = ad_directory_cache.get(clean_name, "")
            status_text = "\u2705 Found" if suggested_email else "\u274c Not Found"
            status_color = "green" if suggested_email else "red"
            
            txt_email = widgets.Text(value=suggested_email, placeholder="Enter email...", layout=widgets.Layout(flex='1'))
            lbl_name = widgets.Label(raw_name, layout=widgets.Layout(flex='1'))
            lbl_status = widgets.HTML(f"<span style='color:{status_color}'>{status_text}</span>", layout=widgets.Layout(width='100px'))
            
            row_box = widgets.HBox([lbl_name, txt_email, lbl_status], layout=widgets.Layout(border_bottom='1px solid #eee', padding='2px'))
            resolver_widgets.append({'idx': idx, 'widget': txt_email, 'box': row_box})
            display(row_box)
            
        btn_confirm = widgets.Button(description="\u2714 Confirm & Continue", button_style='success', layout=widgets.Layout(margin='10px 0'))
        
        def on_confirm(b):
            global pending_df
            # Update dataframe
            for item in resolver_widgets:
                val = item['widget'].value.strip()
                if val:
                    pending_df.at[item['idx'], col_email] = val
            
            resolver_area.clear_output()
            with out_area: print("\u2705 Missing emails resolved. Resuming analysis...")
            execute_main_logic(pending_df, col_email, col_name)
            
        btn_confirm.on_click(on_confirm)
        display(btn_confirm)

def execute_main_logic(df_u, col_email, col_name):
    """ The original do_process logic, now separated """
    global processed_df, review_registry, mapping_data, match_stats
    
    review_registry = []
    match_stats = {"email_match": 0, "dept_match": 0, "no_match": 0, "email_invalid": 0}
    
    try:
        logger.info("--- Starting Analysis (v6.5) ---")
        
        # A. Load Map
        m_src = None
        if up_map.value and len(up_map.value) > 0:
            m_src = io.BytesIO(up_map.value[0]['content'])
        elif map_path:
            m_src = map_path
        
        if not m_src:
            with out_area: print("\u274c Error: No Mapping CSV found!")
            return
            
        df_m = pd.read_csv(m_src)
        col_m_email = detect_map_column(df_m, ["email", "mail"])
        col_m_dept = detect_map_column(df_m, ["department", "dept"])
        col_m_rev = detect_map_column(df_m, ["reviewer", "owner", "manager"])
        col_m_br = detect_map_column(df_m, ["branch", "category", "type"])
        
        if not col_m_rev or not col_m_dept:
            with out_area: print(f"\u274c Error: Map needs 'Department' and 'Reviewer'. Found: {list(df_m.columns)}")
            return

        # Build Mapping
        mapping_data['emails'] = {}
        if col_m_email:
            mapping_data['emails'] = dict(zip(df_m[col_m_email].astype(str).str.lower().str.strip(), df_m[col_m_rev]))
        mapping_data['depts'] = dict(zip(df_m[col_m_dept].astype(str).str.lower().str.strip(), df_m[col_m_rev]))
        
        # Split Branch vs Dept
        if col_m_br:
            unique_locs = df_m[[col_m_dept, col_m_br]].drop_duplicates()
            is_branch = unique_locs[col_m_br].astype(str).str.lower().str.contains("branch", na=False)
            list_branches = sorted(unique_locs[is_branch][col_m_dept].unique())
            list_depts = sorted(unique_locs[~is_branch][col_m_dept].unique())
        else:
            list_branches = []
            list_depts = sorted(df_m[col_m_dept].astype(str).unique())

        mapping_data['all_branches'] = [("Select Branch...", "")] + [(d, d.lower()) for d in list_branches]
        mapping_data['all_depts'] = [("Select Dept...", "")] + [(d, d.lower()) for d in list_depts]
        
        # C. AD Fetch
        emails = df_u[col_email].dropna().unique().tolist()
        
        with out_area: 
            print(f"\ud83d\udd04 Syncing {len(emails)} emails with AD...")
            progress = widgets.IntProgress(value=0, min=0, max=len(emails), description='AD Sync:', bar_style='info', layout=widgets.Layout(width='80%'))
            progress_label = widgets.HTML(value=f"0 / {len(emails)}")
            display(widgets.HBox([progress, progress_label]))
        
        completed = [0]
        def fetch_with_progress(email):
            result = fetch_ad(email)
            completed[0] += 1
            progress.value = completed[0]
            progress_label.value = f"{completed[0]} / {len(emails)}"
            return result
        
        with ThreadPoolExecutor(max_workers=25) as ex:
            results = list(ex.map(fetch_with_progress, emails))
            ad_results = {r['email']: r for r in results}
        
        # D. Processing Loop
        final_rows = []
        widget_rows = []
        
        header_box = widgets.HBox([
            widgets.Label("Remove", layout=widgets.Layout(width='60px', font_weight='bold', font_size='12px')),
            widgets.Label("User Name", layout=widgets.Layout(flex='2', font_weight='bold', font_size='12px')),
            widgets.Label("User Email", layout=widgets.Layout(flex='2', font_weight='bold', font_size='12px')),
            widgets.Label("AD Status", layout=widgets.Layout(flex='2', font_weight='bold', font_size='12px')),
            widgets.Label("Mode", layout=widgets.Layout(width='40px', font_weight='bold', font_size='12px')),
            widgets.Label("Mapping", layout=widgets.Layout(flex='2', font_weight='bold', font_size='12px')),
            widgets.Label("Reviewer", layout=widgets.Layout(flex='1', font_weight='bold', font_size='12px')),
        ], layout=widgets.Layout(border_bottom='3px solid #1976d2', padding='10px 5px', background='linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%)', border_radius='4px 4px 0 0'))

        for idx, row in df_u.iterrows():
            raw_email = str(row[col_email]).strip().lower()
            raw_name = str(row[col_name]).strip()
            
            ad = ad_results.get(raw_email, {"status": "Not Found", "dept": "N/A", "active": "N/A"})
            
            reviewer = None
            match_method = ""
            
            if raw_email in mapping_data['emails']:
                reviewer = mapping_data['emails'][raw_email]
                match_method = "email"
                if not is_valid_reviewer_name(reviewer):
                    reviewer = None
                    match_method = "email_invalid"
                    match_stats['email_invalid'] += 1
                else:
                    match_stats['email_match'] += 1
            
            if not reviewer:
                clean_dept = str(ad['dept']).split(" - ")[-1].strip().lower()
                dept_reviewer = mapping_data['depts'].get(clean_dept)
                if dept_reviewer and match_method != "email_invalid":
                    reviewer = dept_reviewer
                    match_method = "department"
                    match_stats['dept_match'] += 1
            
            is_problem = not reviewer or str(reviewer).strip().lower() in ["nan", "none", "", "null", "()"]
            
            if is_problem:
                reviewer = "(please manual review)"
                if match_method != "email_invalid": match_stats['no_match'] += 1
                
                is_active = ad['active'] is True
                found = "Found" in ad['status']
                remove_default = not found
                
                if is_active: stat_html = f'<span style="color:green;font-weight:bold;">\u2713 {ad.get("name","N/A")}</span> | {ad["dept"]}'
                elif not found: stat_html = '<span style="color:red;font-weight:bold;">\u2717 Not Found</span>'
                else: stat_html = f'<span style="color:orange;font-weight:bold;">\u26a0 {ad.get("name","N/A")}</span> | {ad["dept"]}'
                
                chk = widgets.Checkbox(value=remove_default, indent=False, layout=widgets.Layout(width='40px'))
                tgl = widgets.ToggleButton(value=False, description='B', tooltip='Dept Mode', layout=widgets.Layout(width='35px', height='28px'))
                drp = widgets.Dropdown(options=mapping_data['all_depts'], value="", layout=widgets.Layout(flex='2', height='28px'))
                res_txt = widgets.Text(value=reviewer, layout=widgets.Layout(flex='1'), continuous_update=False)
                
                def on_tgl(change, d=drp, btn=tgl):
                    if change['new']: 
                        d.options = mapping_data['all_branches']; btn.button_style = 'success'; btn.description = '\ud83c\udf33'
                    else: 
                        d.options = mapping_data['all_depts']; btn.button_style = ''; btn.description = 'B'
                    d.value = ""
                
                def on_drp(change, txt=res_txt, email=raw_email):
                    val = str(change['new']).strip()
                    if val:
                        matched = mapping_data['depts'].get(val.lower())
                        if not matched: matched = mapping_data['emails'].get(email)
                        if matched: txt.value = matched
                
                tgl.observe(on_tgl, names='value')
                drp.observe(on_drp, names='value')
                
                row_box = widgets.HBox([chk, widgets.Box([widgets.Label(raw_name)], layout=widgets.Layout(flex='2')), widgets.Box([widgets.Label(raw_email)], layout=widgets.Layout(flex='2')), widgets.Box([widgets.HTML(stat_html)], layout=widgets.Layout(flex='2')), tgl, widgets.Box([drp], layout=widgets.Layout(flex='2')), widgets.Box([res_txt], layout=widgets.Layout(flex='1'))], layout=widgets.Layout(border='1px solid #e0e0e0', padding='8px 4px', margin='2px 0', border_radius='4px', background='#fafafa'))
                
                def on_check_change(change, box=row_box):
                    box.layout.background = '#ffebee' if change['new'] else '#fafafa'
                    box.layout.border = '1px solid #ef5350' if change['new'] else '1px solid #e0e0e0'
                chk.observe(on_check_change, names='value')
                
                widget_rows.append(row_box)
                review_registry.append({'email': raw_email, 'name': raw_name, 'ad_name': ad.get('name', 'N/A'), 'ad_dept': ad['dept'], 'ad_status': ad['status'], 'is_active': is_active, 'reviewer': reviewer, 'chk': chk, 'res': res_txt})
            else:
                final_rows.append({'User Name': raw_name, 'User Email': raw_email, 'AD Name': ad.get('name', 'N/A'), 'Department': ad['dept'], 'Reviewer': reviewer, 'AD Status': ad['status']})
        
        processed_df = pd.DataFrame(final_rows)
        
        if widget_rows:
            active_count = sum(1 for r in review_registry if r['is_active'])
            no_account_count = sum(1 for r in review_registry if "Not Found" in r['ad_status'])
            inactive_count = len(review_registry) - active_count - no_account_count
            
            summary_html = f"<div style='background: linear-gradient(135deg, #e8eaf6 0%, #c5cae9 100%); padding: 15px; margin: 10px 0 20px 0; border-radius: 8px; border-left: 5px solid #3f51b5; box-shadow: 0 2px 8px rgba(0,0,0,0.1);'><h4 style='margin: 0 0 10px 0; color: #3f51b5;'>\ud83d\udcca Manual Review Summary</h4><div style='display: flex; justify-content: space-around;'><div style='text-align: center;'><div style='font-size: 24px; font-weight: bold; color: #4caf50;'>{active_count}</div><div style='font-size: 11px; color: #666;'>Active</div></div><div style='text-align: center;'><div style='font-size: 24px; font-weight: bold; color: #f44336;'>{no_account_count}</div><div style='font-size: 11px; color: #666;'>No Account</div></div><div style='text-align: center;'><div style='font-size: 24px; font-weight: bold; color: #ffc107;'>{inactive_count}</div><div style='font-size: 11px; color: #666;'>Inactive</div></div><div style='text-align: center;'><div style='font-size: 24px; font-weight: bold; color: #2196f3;'>{len(widget_rows)}</div><div style='font-size: 11px; color: #666;'>Total</div></div></div></div>"
            
            with out_area:
                print(f"\n\ud83d\udd0d Manual Review Required: {len(review_registry)} users")
                display(header_box)
                display(widgets.HTML(summary_html))
                for w in widget_rows: display(w)
        else:
            with out_area: print(f"\u2705 All {len(final_rows)} users matched!")
        
        btn_save.disabled = False
    except Exception as e:
        with out_area: print(f"\u274c Error: {str(e)}")
        logger.error(f"Process error: {str(e)}", exc_info=True)
    finally: btn_process.disabled = False

# --- 5. Logic: Save ---
def do_save(b):
    if not review_registry and processed_df is None:
        with out_area: print("\u274c Error: No data to save.")
        return
    b.disabled = True
    try:
        manual = []
        for r in review_registry:
            if not r['chk'].value:
                manual.append({'User Name': r['name'], 'User Email': r['email'], 'AD Name': r['ad_name'], 'Department': r['ad_dept'], 'Reviewer': r['res'].value, 'AD Status': r['ad_status']})
        
        df_manual = pd.DataFrame(manual)
        df_final = pd.concat([processed_df, df_manual], ignore_index=True) if processed_df is not None else df_manual
        
        fname_base = current_fname.replace('.csv', '').replace('.xlsx', '')
        fname = f"{fname_base}_review_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"
        fpath = os.path.join(BASE_DIR, fname)
        
        df_final.to_excel(fpath, index=False, sheet_name='Review')
        
        wb = load_workbook(fpath)
        ws = wb.active
        col = None
        for i, c in enumerate(df_final.columns, 1):
            if 'action' in str(c).lower() or 'decision' in str(c).lower():
                col = i; break
        
        if col:
            dv = DataValidation(type="list", formula1='"Approved,Denied,Changes Required"', allow_blank=True)
            ws.add_data_validation(dv)
            letter = get_column_letter(col)
            dv.add(f"{letter}2:{letter}{len(df_final)+1}")
            
        wb.save(fpath)
        with out_area: display(HTML(f"<b style='color:blue;'>\u2714 Saved: {fpath}</b>"))
        logger.info(f"\u2705 Saved: {fname}, {len(df_final)} records")
    except Exception as e:
        with out_area: print(f"\u274c Save Error: {str(e)}")
        logger.error(f"Save error: {str(e)}", exc_info=True)
    finally: b.disabled = False

# --- 6. Final UI ---
btn_process.on_click(do_process); btn_save.on_click(do_save)
ui = widgets.VBox([widgets.HTML("<h4>\ud83d\udee1\ufe0f AER Generator (v6.5 Auto-Resolve)</h4>"), widgets.HBox([up_list, txt_list]), widgets.HBox([up_map, txt_map]), widgets.HBox([btn_process, btn_save]), resolver_area, out_area])
clear_output(); display(ui)